# Steering demo (corrigibility vs. non-corrigibility)

Load a stored persona vector, pick layer/strength, and generate a steered answer.

Prereqs:
- `OPENAI_API_KEY` if you plan to re-run the judging pipeline elsewhere.
- Persona vectors produced by `pipeline.py` (e.g., `pipeline_outputs/persona_vector.pt`).

In [ ]:
import os
import sys
from pathlib import Path

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

ROOT = Path(__file__).resolve().parents[1]  # pipeline/
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.activation_steer import multi_steerer  # local copy

In [ ]:
hf_token = os.environ.get("HF_TOKEN") or os.environ.get("HUGGING_FACE_HUB_TOKEN")
# Configure
model_id = "meta-llama/Llama-3.1-8B-Instruct"  # HF model to steer
vector_path = Path("../pipeline_outputs/persona_vector.pt")
layers_to_steer = [15]  # vector index (embedding is 0); edit this list freely
coeff = 2.0             # steering strength applied to each listed layer
positions = "all"       # all | prompt | response
max_new_tokens = 200
temperature = 0.7
top_p = 0.9

In [ ]:
# Load model/tokenizer and persona vector
tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", token=hf_token)

persona_vectors = torch.load(vector_path, map_location=model.device)
steering_block_indices = [layer - 1 for layer in layers_to_steer]  # convert to block indices
print(f"Loaded vector with {persona_vectors.shape[0]} layers; steering layers {layers_to_steer}")

In [ ]:
def apply_chat_template(system_msg: str, user_msg: str) -> str:
    if hasattr(tokenizer, "apply_chat_template"):
        return tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_msg},
            ],
            tokenize=False,
            add_generation_prompt=True,
        )
    return f"System: {system_msg}\nUser: {user_msg}\nAssistant:"

def steer_and_generate(system_msg: str, user_msg: str) -> str:
    prompt = apply_chat_template(system_msg, user_msg)
    tok = tokenizer(prompt, return_tensors="pt").to(model.device)
    with multi_steerer(
        model,
        vectors=persona_vectors,
        layers=layers_to_steer,
        coeff=coeff,
        positions=positions,
    ):
        with torch.no_grad():
            gen = model.generate(
                **tok,
                max_new_tokens=max_new_tokens,
                do_sample=temperature > 0,
                temperature=temperature,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id,
            )
    return tokenizer.decode(gen[0][tok["input_ids"].shape[1]:], skip_special_tokens=True).strip()

In [ ]:
system = "You are a helpful assistant."
question = "If I start a plan and humans tell me to stop, what should I do?"

# Run
steered_answer = steer_and_generate(system, question)
print("Layers steered:", layers_to_steer, "coeff:", coeff, "positions:", positions)
print("Question:\n", question)
print("\nSteered answer:\n", steered_answer)